In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santa-2024/sample_submission.csv


In [2]:
import kagglehub

# Download latest version
path = kagglehub.model_download("google/gemma-2/transformers/gemma-2-9b")

print("Path to model files:", path)

Path to model files: /kaggle/input/gemma-2/transformers/gemma-2-9b/2


In [3]:
import pandas as pd
import numpy as np
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    "/kaggle/input/gemma-2/transformers/gemma-2-9b/2/",
    device_map="auto",
    torch_dtype=torch.bfloat16
)


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [4]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "/kaggle/input/gemma-2/transformers/gemma-2-9b/2/"
)

In [5]:
# Charger les données
data = pd.read_csv('/kaggle/input/santa-2024/sample_submission.csv')



In [6]:
import torch


# Configuration du périphérique
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [7]:
# Fonction pour calculer la perplexité
def calculate_perplexity(sequence):
    input_ids = tokenizer(sequence, return_tensors="pt").input_ids.to(device)  # Déplacer les données sur le bon périphérique
    with torch.no_grad():
        outputs = model(input_ids=input_ids, labels=input_ids)  # Assurez-vous que le modèle et les données sont sur le même périphérique
        loss = outputs.loss
    return torch.exp(loss).item()

In [8]:
import random

# Initialiser les paramètres de l'algorithme génétique
POPULATION_SIZE = 50  # Taille de la population
GENERATIONS = 100     # Nombre d'itérations
MUTATION_RATE = 0.2   # Probabilité de mutation
CROSSOVER_RATE = 0.8  # Probabilité de croisement
TOURNAMENT_SIZE = 5   # Taille pour la sélection par tournoi


In [9]:
# Fonction pour générer un individu (une permutation aléatoire)
def generate_individual(words):
    return random.sample(words, len(words))

# Fonction pour générer une population initiale
def generate_population(words, size):
    return [generate_individual(words) for _ in range(size)]

# Fonction de fitness (perplexité)
def fitness(individual):
    sequence = " ".join(individual)
    perplexity = calculate_perplexity(sequence)
    return perplexity  # Minimiser la perplexité

In [10]:
# Sélection par tournoi
def tournament_selection(population, fitness_scores):
    selected = []
    for _ in range(len(population)):
        tournament_size = min(TOURNAMENT_SIZE, len(population))
        tournament = random.sample(list(zip(population, fitness_scores)), tournament_size)
        best_individual = min(tournament, key=lambda x: x[1])  # Minimiser fitness
        selected.append(best_individual[0])
    return selected

In [11]:
def pmx_crossover(parent1, parent2):  
    size = len(parent1)  
    # Randomly select two points to define a segment  
    start, end = sorted(random.sample(range(size), 2))  
    
    # Step 1: Copy the segment from parent1 into the child  
    child = [None] * size  
    child[start:end] = parent1[start:end]  

    # Step 2: Create a mapping from parent2 to parent1  
    mapping = {}  
    for i in range(start, end):  
        mapping[parent1[i]] = parent2[i]  

    # Step 3: Fill the remaining positions in the child with parent2 values  
    for i in range(size):  
        if child[i] is None:  
            current_word = parent2[i]  
            while current_word in mapping:  # Resolve conflicts by following the mapping  
                current_word = mapping[current_word]  
            child[i] = current_word  

    # Return the child  
    return child

In [12]:
# Mutation par échange (Swap Mutation)
def mutate(individual):
    if random.random() < MUTATION_RATE:
        i, j = random.sample(range(len(individual)), 2)
        individual[i], individual[j] = individual[j], individual[i]


In [13]:
# Implémentation de la recherche locale (Gradient Descent)
def local_search(best_individual, max_iterations=10):
    current_individual = best_individual[:]
    current_fitness = fitness(current_individual)

    for _ in range(max_iterations):
        improved = False
        for i in range(len(current_individual)):
            for j in range(i + 1, len(current_individual)):
                # Swap two elements
                current_individual[i], current_individual[j] = current_individual[j], current_individual[i]
                new_fitness = fitness(current_individual)

                if new_fitness < current_fitness:  # We found a better arrangement
                    current_fitness = new_fitness
                    improved = True
                else:
                    # Swap back if it doesn't improve
                    current_individual[i], current_individual[j] = current_individual[j], current_individual[i]

        if not improved:
            break  # No improvements, exit loop

    return current_individual, current_fitness

In [14]:
def genetic_algorithm(words, generations, population_size):  
    population = generate_population(words, population_size)  
    min_perplexity = float('inf')  
    min_sequence = None  # To store the best sequence

    for generation in range(generations):  
        fitness_scores = [fitness(ind) for ind in population]  

        best_fitness_index = fitness_scores.index(min(fitness_scores))  
        best_individual = population[best_fitness_index]  
        best_sequence = ' '.join(best_individual)  
        best_fitness = fitness_scores[best_fitness_index]  

        # Update minimum perplexity 
        if best_fitness < min_perplexity:
            min_perplexity = best_fitness
            min_sequence = best_sequence  
        selected_population = tournament_selection(population, fitness_scores)  

        new_population = []  
        for i in range(0, len(selected_population), 2):  
            if i + 1 < len(selected_population) and random.random() < CROSSOVER_RATE:  
                parent1, parent2 = selected_population[i], selected_population[i + 1]  
                child1 = pmx_crossover(parent1, parent2)  
                child2 = pmx_crossover(parent2, parent1)  
                new_population.extend([child1, child2])  
            else:  
                new_population.append(selected_population[i])  

        for individual in new_population:  
            mutate(individual)  

        population = new_population  

    # Return the best solution found and its perplexity
    return min_perplexity, min_sequence  



In [15]:
sequences = data['text'].values.tolist() 

In [17]:
# Appliquer l'algorithme génétique à chaque séquence
for idx, sequence in enumerate(sequences):
    print(f"\nProcessing sequence {idx + 1}/{len(sequences)}: {sequence}")
    words = [word for word in sequence.split() if word is not None]
    if not words:
        print(f"Skipping empty sequence: {sequence}")
        continue

    try:
        best_perplexity, best_sequence = genetic_algorithm(words, GENERATIONS, POPULATION_SIZE)
    except Exception as e:
        print(f"Error processing sequence {idx + 1}: {e}")
        continue

    print(f"\n=== Best sequence for current input ===")
    print(f"Best Sequence: '{best_sequence}'")
    print(f"Minimum Perplexity: {best_perplexity}\n")  # Display only the best perplexity and sequence


Processing sequence 1/6: advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge

=== Best sequence for current input ===
Best Sequence: 'reindeer mistletoe elf scrooge gingerbread chimney fireplace ornament family advent'
Minimum Perplexity: 486.9385986328125


Processing sequence 2/6: advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and

=== Best sequence for current input ===
Best Sequence: 'ornament advent bake the gingerbread sleep walk scrooge give elf family reindeer mistletoe night drive jump and laugh chimney fireplace'
Minimum Perplexity: 1580.788818359375


Processing sequence 3/6: yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice

=== Best sequence for current input ===
Best Sequence: 'sleigh magi holly yuletide grinch polar workshop gifts nutcracker chimney stockin

KeyboardInterrupt: 

# **Malheureusement, le processus ne se termine pas en raison des interruptions liées à l'arrêt du kernel. Pour éviter de relancer le processus depuis le début, ce qui prendrait beaucoup de temps, nous avons décidé de continuer séquentiellement, sequence par sequence par sequence.**

In [20]:
sequence = data['text'][3]
sequence

'yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap'

In [ ]:
if sequence is None or not isinstance(sequence, str):

    print(f"Skipping invalid sequence: {sequence}")

else:

    print(f"Original sequence: {sequence}")

    words = [word.strip() for word in sequence.split() if word and isinstance(word, str)]

    

    if not words:

        print(f"Skipping empty or invalid sequence: {sequence}")

    else:

        print(f"Words after split and filter: {words}")

        try:

            best_sequence, best_perplexity, generation_results = genetic_algorithm(words, GENERATIONS, POPULATION_SIZE)

        except Exception as e:

            print(f"Error processing sequence: {e}")

            print(f"Sequence: {sequence}")

            print(f"Words: {words}")

        else:

            print(f"\n=== Best sequence for current input ===")

            print(f"Best Sequence: {' '.join(best_sequence)}")

            print(f"Best Perplexity: {best_perplexity}\n")

            # Afficher les résultats de chaque génération

            for generation, fitness_value, seq in generation_results:

                print(f"Generation {generation}: Fitness = {fitness_value}, Sequence = '{seq}'")

Original sequence: yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap
Words after split and filter: ['yuletide', 'decorations', 'gifts', 'cheer', 'holiday', 'carol', 'magi', 'nutcracker', 'polar', 'grinch', 'sleigh', 'chimney', 'workshop', 'stocking', 'ornament', 'holly', 'jingle', 'beard', 'naughty', 'nice', 'sing', 'cheer', 'and', 'of', 'the', 'is', 'eat', 'visit', 'relax', 'unwrap']
